In [1]:
# @hidden_cell
# The project token is an authorization token that is used to access project resources like data sources, connections, and used by platform APIs.
from project_lib import Project
project = Project(project_id='aa55489f-5f1f-417a-9f33-5bd5457b2fb5', project_access_token='p-9d5f6e0d91bd4e98574f0d38bb268ddf3b17435a')
pc = project.project_context
with open("geocoder-8.x-3.0-alpha1.tar.gz","wb") as f:
    f.write(project.get_file("geocoder-8.x-3.0-alpha1.tar.gz").read())

 # Install the library
!pip install geocoder

     |████████████████████████████████| 102kB 10.9MB/s ta 0:00:01


Build a dataframe of the postal code of each neighborhood along with the borough name and neighborhood name in Toronto.

# Importing Libraries

In [2]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

print("Libraries imported.")

Libraries imported.


# Creating the Dataset

Getting data from wikipedia

In [3]:
data = requests.get('https://commons.wikimedia.org/wiki/Category:Neighborhoods_in_Florence').text

# parse data from the html into a beautifulsoup object
soup = BeautifulSoup(data, 'html.parser')

# create a list to store table data
neighborhoodList = []

# Using Soup to better analyze it

In [4]:
# append the data into the list
for row in soup.find_all("div", class_="mw-category")[0].findAll("li"):
    neighborhoodList.append(row.text)

In [5]:
# create a new DataFrame from the list
fl_df = pd.DataFrame({"Neighborhood": neighborhoodList})

fl_df = fl_df.head(33)
fl_df.shape

(33, 1)

# Modifying the data making it looks better/Clean the data

In [6]:
import re
new_df = []
for index, row in fl_df.iterrows():
    name = row["Neighborhood"][3:]
    almost = re.sub("[\(\[].*?[\)\]]", "", name)
    enough = re.sub("\\u200e", "", almost)
    last = re.sub(" ", "", enough)
    new_df.append(last)
    
new_df

['Arcetri',
 'Argingrosso',
 'Bellariva',
 'Bellosguardo',
 'Brozzi',
 'CampodiMarte',
 'Careggi',
 'Castello',
 'Corbignano',
 'Coverciano',
 'Galluzzo',
 'Gavinana',
 'Isolotto',
 'SanJacopino',
 'LePiagge',
 'MonteOliveto',
 'Monterinaldi',
 'Monteripaldi',
 'Montughi',
 'Novoli',
 'Peretola',
 'Rifredi',
 'Rovezzano',
 'Ruffignano',
 'Salviatino',
 'Serpiolle',
 'Settignano',
 'Soffiano',
 'Sorgane',
 'Statuto',
 'Trespiano',
 'Varlungo',
 'Villamagna']

In [7]:
fl_df = pd.DataFrame({"Neighborhood": new_df})
fl_df.head()

,Neighborhood
0,Arcetri
1,Argingrosso
2,Bellariva
3,Bellosguardo
4,Brozzi


# Making the last print of the size of the dataframe

In [8]:
import geocoder
# define a function to get coordinates
def get_latlng(neighborhood):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Florence, Italy'.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords
# call the function to get the coordinates, store in a new list using list comprehension
coords = [ get_latlng(neighborhood) for neighborhood in fl_df["Neighborhood"].tolist() ]
coords

[[43.745870000000025, 11.252940000000024],
 [43.78716440669812, 11.181306229669698],
 [43.76885000000004, 11.286600000000021],
 [43.68735000000004, 11.501880000000028],
 [43.79399259103889, 11.171280051944066],
 [41.55311000000006, 12.639580000000024],
 [43.81217000000004, 11.248470000000054],
 [43.77901000000003, 11.094570025535289],
 [43.7885535609057, 11.313330717595075],
 [43.78170994806609, 11.308929922681662],
 [43.73675000000003, 11.226870000000076],
 [43.76179993060772, 11.288770011108552],
 [43.78774132155207, 11.199172625208956],
 [43.783930000000055, 11.240640000000042],
 [43.791709921086685, 11.172010022217048],
 [43.76812000000007, 11.236920000000055],
 [43.80946162293606, 11.27898569625404],
 [43.74501968892768, 11.258185890597561],
 [43.79381876298496, 11.255358183034774],
 [43.72512092053536, 11.439757380803036],
 [43.79647624722334, 11.194446059951275],
 [43.80260000000004, 11.213290000000029],
 [43.77087423018739, 11.313201760318837],
 [43.831108923836375, 11.24719119

In [9]:
# create temporary dataframe to populate the coordinates into Latitude and Longitude
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])

# merge the coordinates into the original dataframe
fl_df['Latitude'] = df_coords['Latitude']
fl_df['Longitude'] = df_coords['Longitude']

# check the neighborhoods and the coordinates
print(fl_df.shape)
fl_df

(33, 3)


,Neighborhood,Latitude,Longitude
0,Arcetri,43.745870,11.252940
1,Argingrosso,43.787164,11.181306
2,Bellariva,43.768850,11.286600
3,Bellosguardo,43.687350,11.501880
4,Brozzi,43.793993,11.171280
5,CampodiMarte,41.553110,12.639580
6,Careggi,43.812170,11.248470
7,Castello,43.779010,11.094570
8,Corbignano,43.788554,11.313331
9,Coverciano,43.781710,11.308930


# Dataset created, now start to use the Locations to build a map

In [10]:
from geopy.geocoders import Nominatim
address = 'Florence, Italy'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Florence are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Florence are 43.7698712, 11.2555757.


In [11]:
!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library


Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    branca-0.3.1               |             py_0          25 KB  conda-forge
    openssl-1.1.1c             |       h516909a_0         2.1 MB  conda-forge
    altair-3.1.0               |           py36_0         724 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    ca-certificates-2019.6.16  |       hecc5488_0         145 KB  conda-forge
    certifi-2019.6.16          |           py36_1         149 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         3.2 MB

The following NEW packages will be 

In [12]:
# create map of Toronto using latitude and longitude values
map_fl = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, neighborhood in zip(fl_df['Latitude'], fl_df['Longitude'], fl_df['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_fl)  
    
map_fl

# Starting to use the Foursquare API now to explore the neighbors

In [13]:
# define Foursquare Credentials and Version
CLIENT_ID = '2QN42EGGXU3AQQLGKPC2KKNMLBSCXVJ0XND4UBYR01FIKJK4' # your Foursquare ID
CLIENT_SECRET = 'WAB4WPRVY1SL1FRTP41BUVVXHUXO3U4W5HQRBHYT50AEJAGP' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 2QN42EGGXU3AQQLGKPC2KKNMLBSCXVJ0XND4UBYR01FIKJK4
CLIENT_SECRET:WAB4WPRVY1SL1FRTP41BUVVXHUXO3U4W5HQRBHYT50AEJAGP


In [14]:
radius = 1000
LIMIT = 100

venues = []

for lat, long, neighborhood in zip(fl_df['Latitude'], fl_df['Longitude'], fl_df['Neighborhood']):
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [15]:
# convert the venues list into a new DataFrame
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = [ 'Neighborhood', 'NeighborhoodLatitude', 'NeighborhoodLongitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(810, 7)


,Neighborhood,NeighborhoodLatitude,NeighborhoodLongitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,Arcetri,43.74587,11.25294,Trattoria Omero,43.747759,11.257891,Italian Restaurant
1,Arcetri,43.74587,11.25294,Villa Le Piazzole,43.742887,11.244517,Bed & Breakfast
2,Arcetri,43.74587,11.25294,Piazzale del Poggio Imperiale,43.751694,11.243958,Plaza
3,Arcetri,43.74587,11.25294,Chalet Fontana,43.754771,11.251457,Pizza Place
4,Arcetri,43.74587,11.25294,Osservatorio Astrofisico di Arcetri,43.746346,11.244157,Planetarium


In [16]:
venues_df.groupby(["VenueCategory"]).count()

,Neighborhood,NeighborhoodLatitude,NeighborhoodLongitude,VenueName,VenueLatitude,VenueLongitude
VenueCategory,,,,,,
Airport,2,2,2,2,2,2
Airport Food Court,2,2,2,2,2,2
Airport Lounge,1,1,1,1,1,1
Airport Service,1,1,1,1,1,1
Airport Terminal,3,3,3,3,3,3
Arts & Crafts Store,1,1,1,1,1,1
Asian Restaurant,1,1,1,1,1,1
Athletics & Sports,1,1,1,1,1,1
Automotive Shop,1,1,1,1,1,1


In [17]:
venues_df['VenueCategory'].unique()

array(['Italian Restaurant', 'Bed & Breakfast', 'Plaza', 'Pizza Place',
       'Planetarium', 'Café', 'Gym Pool', 'Shopping Mall', 'Nightclub',
       'Train Station', 'Pet Store', 'Grocery Store', 'Supermarket',
       'Hotel', 'Ice Cream Shop', 'Volleyball Court', 'Wine Bar',
       'Music Venue', 'Chinese Restaurant', 'Event Space',
       'Gym / Fitness Center', 'Pub', 'Food Truck', 'Park', 'Tea Room',
       'Sandwich Place', 'Bakery', 'Pool', 'Food Court',
       'Japanese Restaurant', 'Concert Hall', 'Restaurant',
       'Motorcycle Shop', 'Burger Joint', 'Fast Food Restaurant',
       'Noodle House', 'Steakhouse', 'Seafood Restaurant',
       'Indie Movie Theater', 'Multiplex', 'Other Great Outdoors',
       'Soccer Field', 'Beer Garden', 'River', 'Ski Area',
       'Toy / Game Store', 'Electronics Store', 'Movie Theater',
       'Rock Club', 'Basketball Court', 'Beer Store', 'Cosmetics Shop',
       'Tennis Court', 'Golf Course', 'Construction & Landscaping',
       'Automotiv

# Analyzing the Four Square data received and modeling the database

In [22]:
#Knowing which are the steakhouses in Florence
ven_steak = venues_df.loc[venues_df['VenueCategory'] == 'Steakhouse']
ven_steak

,Neighborhood,NeighborhoodLatitude,NeighborhoodLongitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
66,Bellariva,43.76885,11.286600,Festa Del Mugello,43.766643,11.295227,Steakhouse
198,Gavinana,43.76180,11.288770,Festa Del Mugello,43.766643,11.295227,Steakhouse
291,SanJacopino,43.78393,11.240640,Chianineria Trattoria Dall Oste Centro,43.776350,11.246560,Steakhouse
784,Varlungo,43.76057,11.303698,Festa Del Mugello,43.766643,11.295227,Steakhouse


In [36]:
# one hot encoding
fl_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
fl_onehot['Neighborhoods'] = venues_df['Neighborhood'] 
fl_onehot['NeighborhoodLatitude'] = venues_df['NeighborhoodLatitude'] 
fl_onehot['NeighborhoodLongitude'] = venues_df['NeighborhoodLongitude'] 

# move neighborhood column to the first column
fixed_columns = [fl_onehot.columns[-1]] + list(fl_onehot.columns[:-1])
fl_onehot = fl_onehot[fixed_columns]

print(fl_onehot.shape)
fl_onehot.head()

(810, 135)


,NeighborhoodLongitude,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Automotive Shop,BBQ Joint,Bakery,Bar,Basketball Court,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Bistro,Boarding House,Bookstore,Breakfast Spot,Buffet,Burger Joint,Bus Station,Café,Campground,Chinese Restaurant,Church,Cocktail Bar,Coffee Shop,College Cafeteria,Concert Hall,Construction & Landscaping,Cosmetics Shop,Cupcake Shop,Deli / Bodega,Dessert Shop,Diner,Electronics Store,Event Space,Fast Food Restaurant,Financial or Legal Service,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Furniture / Home Store,Garden,Gastropub,Golf Course,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Historic Site,History Museum,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Italian Restaurant,Japanese Restaurant,Jewelry Store,Korean Restaurant,Lake,Light Rail Station,Lounge,Market,Mexican Restaurant,Middle Eastern Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Motorcycle Shop,Movie Theater,Multiplex,Museum,Music Venue,National Park,Nightclub,Noodle House,Other Great Outdoors,Paper / Office Supplies Store,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Planetarium,Playground,Plaza,Pool,Pub,Rental Car Location,Resort,Restaurant,River,Rock Club,Roof Deck,Sandwich Place,Scenic Lookout,Science Museum,Sculpture Garden,Seafood Restaurant,Shopping Mall,Ski Area,Soccer Field,Soccer Stadium,Sporting Goods Shop,Stables,Steakhouse,Supermarket,Sushi Restaurant,Tea Room,Tennis Court,Theater,Toy / Game Store,Trail,Train Station,Trattoria/Osteria,Tuscan Restaurant,Vacation Rental,Vegetarian / Vegan Restaurant,Volleyball Court,Wine Bar,Wine Shop,Winery,Neighborhoods,NeighborhoodLatitude
0,11.25294,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Arcetri,43.74587
1,11.25294,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Arcetri,43.74587
2,11.25294,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Arcetri,43.74587
3,11.25294,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Arcetri,43.74587
4,11.25294,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Arcetri,43.74587


In [37]:
fl_grouped = fl_onehot.groupby(["Neighborhoods", 'NeighborhoodLongitude', 'NeighborhoodLatitude']).mean().reset_index()

print(fl_grouped.shape)
fl_grouped

(33, 135)


,Neighborhoods,NeighborhoodLongitude,NeighborhoodLatitude,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Automotive Shop,BBQ Joint,Bakery,Bar,Basketball Court,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Bistro,Boarding House,Bookstore,Breakfast Spot,Buffet,Burger Joint,Bus Station,Café,Campground,Chinese Restaurant,Church,Cocktail Bar,Coffee Shop,College Cafeteria,Concert Hall,Construction & Landscaping,Cosmetics Shop,Cupcake Shop,Deli / Bodega,Dessert Shop,Diner,Electronics Store,Event Space,Fast Food Restaurant,Financial or Legal Service,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Furniture / Home Store,Garden,Gastropub,Golf Course,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Historic Site,History Museum,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Italian Restaurant,Japanese Restaurant,Jewelry Store,Korean Restaurant,Lake,Light Rail Station,Lounge,Market,Mexican Restaurant,Middle Eastern Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Motorcycle Shop,Movie Theater,Multiplex,Museum,Music Venue,National Park,Nightclub,Noodle House,Other Great Outdoors,Paper / Office Supplies Store,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Planetarium,Playground,Plaza,Pool,Pub,Rental Car Location,Resort,Restaurant,River,Rock Club,Roof Deck,Sandwich Place,Scenic Lookout,Science Museum,Sculpture Garden,Seafood Restaurant,Shopping Mall,Ski Area,Soccer Field,Soccer Stadium,Sporting Goods Shop,Stables,Steakhouse,Supermarket,Sushi Restaurant,Tea Room,Tennis Court,Theater,Toy / Game Store,Trail,Train Station,Trattoria/Osteria,Tuscan Restaurant,Vacation Rental,Vegetarian / Vegan Restaurant,Volleyball Court,Wine Bar,Wine Shop,Winery
0,Arcetri,11.252940,43.745870,0.000000,0.000000,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.285714,0.0000,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.142857,0.000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.142857,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.142857,0.142857,0.0,0.142857,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.00,0.00
1,Argingrosso,11.181306,43.787164,0.000000,0.000000,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.111111,0.000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.111111,0.000000,0.000000,0.111111,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.111111,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.111111,0.000000,0.000000,0.0,0.000000,0.000000,0.111111,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000,0.000000,0.000000,0.111111,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.111111,0.000000,0.000000,0.000000,0.000000,0.000000,0.0

# Creating the clusterization based in Venues Categories, quantity of venues and distances from neighborhoods

In [82]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
areaColumns = ['Neighborhoods', 'NeighborhoodLatitude', 'NeighborhoodLongitude']
freqColumns = []
for ind in np.arange(num_top_venues):
    try:
        freqColumns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        freqColumns.append('{}th Most Common Venue'.format(ind+1))
columns = areaColumns+freqColumns

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhoods'] = fl_grouped['Neighborhoods']
neighborhoods_venues_sorted['NeighborhoodLatitude'] = fl_grouped['NeighborhoodLatitude']
neighborhoods_venues_sorted['NeighborhoodLongitude'] = fl_grouped['NeighborhoodLongitude']

for ind in np.arange(fl_grouped.shape[0]):
    row_categories = fl_grouped.iloc[ind, :].iloc[3:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    neighborhoods_venues_sorted.iloc[ind, 3:] = row_categories_sorted.index.values[0:num_top_venues]

# neighborhoods_venues_sorted.sort_values(freqColumns, inplace=True)
print(neighborhoods_venues_sorted.shape)
neighborhoods_venues_sorted

(33, 13)


,Neighborhoods,NeighborhoodLatitude,NeighborhoodLongitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Arcetri,43.745870,11.252940,Bed & Breakfast,Italian Restaurant,Planetarium,Pizza Place,Café,Plaza,Historic Site,Gym Pool,Deli / Bodega,Hotel Bar
1,Argingrosso,43.787164,11.181306,Italian Restaurant,Train Station,Shopping Mall,Nightclub,Supermarket,Café,Grocery Store,Gym Pool,Pet Store,Diner
2,Bellariva,43.768850,11.286600,Italian Restaurant,Pizza Place,Café,Hotel,Ice Cream Shop,Supermarket,Pub,Grocery Store,Wine Bar,Music Venue
3,Bellosguardo,43.687350,11.501880,Italian Restaurant,Tennis Court,Pizza Place,Cosmetics Shop,Hotel,Hostel,Cupcake Shop,Deli / Bodega,Dessert Shop,Ice Cream Shop
4,Brozzi,43.793993,11.171280,Japanese Restaurant,Shopping Mall,Nightclub,Train Station,Grocery Store,Winery,Financial or Legal Service,Food,Flower Shop,Flea Market
5,CampodiMarte,41.553110,12.639580,Automotive Shop,Construction & Landscaping,Golf Course,Train Station,Winery,Food & Drink Shop,Food,Flower Shop,Flea Market,Financial or Legal Service
6,Careggi,43.812170,11.248470,Italian Restaurant,Financial or Legal Service,Deli / Bodega,Hotel,Café,Food Court,Food & Drink Shop,Food,Flower Shop,Flea Market
7,Castello,43.779010,11.094570,Pizza Place,Italian Restaurant,Park,Soccer Field,Café,Bed & Breakfast,Breakfast Spot,Grocery Store,Buffet,Lake
8,Corbignano,43.788554,11.313331,Italian Restaurant,Trattoria/Osteria,Restaurant,Park,Café,Plaza,Trail,Food Court,Hotel,Gym / Fitness Center
9,Coverciano,43.781710,11.308930,Hotel,Italian Restaurant,Diner,Soccer Field,Soccer Stadium,Gym / Fitness Center,Museum,Dessert Shop,Resort,Deli / Bodega


In [84]:
# set number of clusters
kclusters = 5

fl_grouped_clustering = fl_grouped.drop(["Neighborhoods", 'NeighborhoodLatitude', 'NeighborhoodLongitude'], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(fl_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([3, 0, 3, 1, 0, 3, 3, 3, 3, 3], dtype=int32)

Meraging the dataset and creating our final dataframe

In [85]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
fl_merged = fl_df.copy()

# add clustering labels
fl_merged["Cluster Labels"] = kmeans.labels_

fl_merged = fl_merged.join(neighborhoods_venues_sorted.drop(["NeighborhoodLatitude", "NeighborhoodLongitude"], 1).set_index("Neighborhoods"), on="Neighborhood")


print(fl_merged.shape)
fl_merged

(33, 14)


,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Arcetri,43.745870,11.252940,3,Bed & Breakfast,Italian Restaurant,Planetarium,Pizza Place,Café,Plaza,Historic Site,Gym Pool,Deli / Bodega,Hotel Bar
1,Argingrosso,43.787164,11.181306,0,Italian Restaurant,Train Station,Shopping Mall,Nightclub,Supermarket,Café,Grocery Store,Gym Pool,Pet Store,Diner
2,Bellariva,43.768850,11.286600,3,Italian Restaurant,Pizza Place,Café,Hotel,Ice Cream Shop,Supermarket,Pub,Grocery Store,Wine Bar,Music Venue
3,Bellosguardo,43.687350,11.501880,1,Italian Restaurant,Tennis Court,Pizza Place,Cosmetics Shop,Hotel,Hostel,Cupcake Shop,Deli / Bodega,Dessert Shop,Ice Cream Shop
4,Brozzi,43.793993,11.171280,0,Japanese Restaurant,Shopping Mall,Nightclub,Train Station,Grocery Store,Winery,Financial or Legal Service,Food,Flower Shop,Flea Market
5,CampodiMarte,41.553110,12.639580,3,Automotive Shop,Construction & Landscaping,Golf Course,Train Station,Winery,Food & Drink Shop,Food,Flower Shop,Flea Market,Financial or Legal Service
6,Careggi,43.812170,11.248470,3,Italian Restaurant,Financial or Legal Service,Deli / Bodega,Hotel,Café,Food Court,Food & Drink Shop,Food,Flower Shop,Flea Market
7,Castello,43.779010,11.094570,3,Pizza Place,Italian Restaurant,Park,Soccer Field,Café,Bed & Breakfast,Breakfast Spot,Grocery Store,Buffet,Lake
8,Corbignano,43.788554,11.313331,3,Italian Restaurant,Trattoria/Osteria,Restaurant,Park,Café,Plaza,Trail,Food Court,Hotel,Gym / Fitness Center
9,Coverciano,43.781710,11.308930,3,Hotel,Italian Restaurant,Diner,Soccer Field,Soccer Stadium,Gym / Fitness Center,Museum,Dessert Shop,Resort,Deli / Bodega


In [87]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(fl_merged['Latitude'], fl_merged['Longitude'], fl_merged['Neighborhood'], fl_merged['Cluster Labels']):
    label = folium.Popup('{} - Cluster {}'.format(poi, cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=['blue', 'white', 'red', 'orange', 'black',],
        fill_opacity=1).add_to(map_clusters)
       
map_clusters

# Understaind and Exploration of Clusters

In [88]:
fl_merged.loc[fl_merged['Cluster Labels'] == 0, fl_merged.columns[[0] + list(range(4, fl_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Argingrosso,Italian Restaurant,Train Station,Shopping Mall,Nightclub,Supermarket,Café,Grocery Store,Gym Pool,Pet Store,Diner
4,Brozzi,Japanese Restaurant,Shopping Mall,Nightclub,Train Station,Grocery Store,Winery,Financial or Legal Service,Food,Flower Shop,Flea Market
13,SanJacopino,Hotel,Pizza Place,Italian Restaurant,Plaza,Café,Bed & Breakfast,Ice Cream Shop,Japanese Restaurant,Trattoria/Osteria,Gym


At cluster 1 we can find the most part of nightclubs, mixed with malls and some food, more residential areas.

In [89]:
fl_merged.loc[fl_merged['Cluster Labels'] == 1, fl_merged.columns[[0] + list(range(4, fl_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,Bellosguardo,Italian Restaurant,Tennis Court,Pizza Place,Cosmetics Shop,Hotel,Hostel,Cupcake Shop,Deli / Bodega,Dessert Shop,Ice Cream Shop
14,LePiagge,Japanese Restaurant,Shopping Mall,Nightclub,Train Station,Grocery Store,Winery,Financial or Legal Service,Food,Flower Shop,Flea Market
16,Monterinaldi,Hotel,Italian Restaurant,Café,Resort,Coffee Shop,Moroccan Restaurant,Scenic Lookout,Flower Shop,Flea Market,Financial or Legal Service
30,Trespiano,Italian Restaurant,Bus Station,Pharmacy,Café,Construction & Landscaping,Airport Terminal,Deli / Bodega,Diner,Electronics Store,Airport Food Court


Tourist areas, with a lot of hotels, restaurants and shopping malls. Incluiding the airport.

In [90]:
fl_merged.loc[fl_merged['Cluster Labels'] == 2, fl_merged.columns[[0] + list(range(4, fl_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
32,Villamagna,Italian Restaurant,Furniture / Home Store,Cupcake Shop,Deli / Bodega,Dessert Shop,Diner,Electronics Store,Event Space,Fast Food Restaurant,Financial or Legal Service


Lonely cluster, this is an isolated neighbor

In [91]:
fl_merged.loc[fl_merged['Cluster Labels'] == 3, fl_merged.columns[[0] + list(range(4, fl_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Arcetri,Bed & Breakfast,Italian Restaurant,Planetarium,Pizza Place,Café,Plaza,Historic Site,Gym Pool,Deli / Bodega,Hotel Bar
2,Bellariva,Italian Restaurant,Pizza Place,Café,Hotel,Ice Cream Shop,Supermarket,Pub,Grocery Store,Wine Bar,Music Venue
5,CampodiMarte,Automotive Shop,Construction & Landscaping,Golf Course,Train Station,Winery,Food & Drink Shop,Food,Flower Shop,Flea Market,Financial or Legal Service
6,Careggi,Italian Restaurant,Financial or Legal Service,Deli / Bodega,Hotel,Café,Food Court,Food & Drink Shop,Food,Flower Shop,Flea Market
7,Castello,Pizza Place,Italian Restaurant,Park,Soccer Field,Café,Bed & Breakfast,Breakfast Spot,Grocery Store,Buffet,Lake
8,Corbignano,Italian Restaurant,Trattoria/Osteria,Restaurant,Park,Café,Plaza,Trail,Food Court,Hotel,Gym / Fitness Center
9,Coverciano,Hotel,Italian Restaurant,Diner,Soccer Field,Soccer Stadium,Gym / Fitness Center,Museum,Dessert Shop,Resort,Deli / Bodega
10,Galluzzo,Grocery Store,Pizza Place,Japanese Restaurant,Playground,Restaurant,Garden,Paper / Office Supplies Store,Supermarket,Bed & Breakfast,Café
11,Gavinana,Café,Hotel,Ice Cream Shop,Pizza Place,Italian Restaurant,Park,Wine Bar,Electronics Store,Pub,Chinese Restaurant
12,Isolotto,Hotel,Pizza Place,Pharmacy,Supermarket,Bus Station,Nightclub,Scenic Lookout,Fast Food Restaurant,Sculpture Garden,Soccer Field


Urban area, most part of the city center is here, most part of the restaurants and hotels are in this area, also the stadium and green areas.

In [92]:
fl_merged.loc[fl_merged['Cluster Labels'] == 4, fl_merged.columns[[0] + list(range(4, fl_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
22,Rovezzano,Pizza Place,Campground,Italian Restaurant,Basketball Court,Gym / Fitness Center,Lounge,Nightclub,Park,Pharmacy,Cocktail Bar


It's the suburb of the city, closest to the trails and with some green area.

# Results

Toscana, the major region where Florence is in, is very well known by the quality of the sheep meat with a large production. Italians are very traditional, they love eat pasta, pizza and even barbecue, but for sheeps, they use to serve it in the plate, as a main meal, with some pasta. In Uruguay and South of Brazil, where the climate is very similar, the sheeps have the same meat quality, but in their culture they eat the meat as a barbecue, performing a very different and extraordinary experience in steakhouses. The main idea of this project was to identify where is the best place to open a specific Steakhouse that could develop again the sheep culture inside italy, and the best results was in neighborhoods of cluster 2, mainly in Bellosguardo, LePiagge and Monterinaldi. Due to the concentration of hotels and the already existance of some restaurants that run away from the traditional pizza and pasta, where people can go and have another good option, that will be the Sheephouse.